# Unit Test Notebook

In [ ]:
from snowflake.snowpark.context import get_active_session

# Enter the Running env
env = 'DEV'

database_name = 'FRED_DB'
schema_name = f'{env}_ANALYTICS'
session = get_active_session()

## Creating The Unit Test Log Table

In [ ]:
CREATE OR REPLACE TABLE FRED_DB.INTEGRATIONS.TEST_LOGS (
    TEST_NAME STRING,
    EXPECTED STRING,
    OUTPUT STRING,
    TEST_STATUS STRING,
    TEST_MESSAGE STRING,
    TEST_TIMESTAMP TIMESTAMP_LTZ DEFAULT CURRENT_TIMESTAMP()
);

## PyTest Functions to Test the defined UDF

In [ ]:
def log_test_result(test_name, expected, output, test_status, test_message):
    """Log test results to the TEST_LOGS table."""
    session.sql(f"""
        INSERT INTO FRED_DB.INTEGRATIONS.TEST_LOGS (TEST_NAME, EXPECTED, OUTPUT, TEST_STATUS, TEST_MESSAGE)
        VALUES ('{test_name}', '{expected}', '{output}', '{test_status}', '{test_message}')
    """).collect()
    
# Example test
def test_calculate_spread():
    try:
        result = session.sql(f"SELECT {database_name}.{schema_name}.calculate_spread(1.88, 0.23)").collect()[0][0]
        assert result == 1.65, f"Expected 1.65, got {result}"
        log_test_result("test_calculate_spread", "1.65", str(result), "PASS", "Spread calculation is correct")
    except Exception as e:
        log_test_result("test_calculate_spread", "1.65", str(result), "FAIL", str(e))

def test_check_spread_status():
    try:
        # Positive spread
        result = session.sql(f"SELECT {database_name}.{schema_name}.check_spread_status(1.65)").collect()[0][0]
        assert result == 'POSITIVE', f"Expected 'POSITIVE', got {result}"
        log_test_result("test_check_spread_status", "POSITIVE", str(result), "PASS", "Spread status calculation is correct")
        
        # Negative spread
        result = session.sql(f"SELECT {database_name}.{schema_name}.check_spread_status(-0.10)").collect()[0][0]
        assert result == 'NEGATIVE', f"Expected 'NEGATIVE', got {result}"
        log_test_result("test_check_spread_status", "NEGATIVE", str(result), "PASS", "Spread status calculation is correct")
        
        # Zero spread
        result = session.sql(f"SELECT {database_name}.{schema_name}.check_spread_status(0.00)").collect()[0][0]
        assert result == 'ZERO', f"Expected 'ZERO', got {result}"
        
        # Log success
        log_test_result("test_check_spread_status", "ZERO", str(result), "PASS", "Spread status calculation is correct")
    except Exception as e:
        # Log failure
        log_test_result("test_check_spread_status", "", "", "FAIL", str(e))


## Running The Tests

In [ ]:
# Running the tests
test_calculate_spread()
test_check_spread_status()